In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup as BS

In [2]:
df = pd.read_csv('cars_urls.csv')
# df = df.drop('Unnamed: 0',axis=1)
# df = df.value_counts().index.to_list()

In [3]:
df = pd.DataFrame(df['url'].unique())
df.columns = ['url']

In [4]:
df

,url
0,https://auto.ru/cars/used/sale/audi/200/109821...
1,https://auto.ru/cars/used/sale/audi/80/1086353...
2,https://auto.ru/cars/used/sale/audi/100/110337...
3,https://auto.ru/cars/used/sale/audi/100/109674...
4,https://auto.ru/cars/used/sale/audi/100/110282...
...,...
11365,https://auto.ru/cars/used/sale/mercedes/w136/1...
11366,https://auto.ru/cars/used/sale/mercedes/w136/1...
11367,https://auto.ru/cars/used/sale/mercedes/w136/1...
11368,https://auto.ru/cars/used/sale/mercedes/w142/1...


In [98]:
test = pd.read_csv('test.csv')

In [99]:
models = [x.lower() for x in test.model_name.unique()]


In [100]:
models

['octavia',
 'superb',
 'octavia_rs',
 'yeti',
 'kodiaq',
 'rapid',
 'fabia',
 'roomster',
 'felicia',
 'karoq',
 'fabia_rs',
 '120',
 '100_series',
 'favorit',
 'popular',
 'q5',
 'r8',
 'q7',
 'q3',
 'a4',
 'a3',
 'a5',
 'a6',
 'rs7',
 's6',
 'rs6',
 'allroad',
 'tt',
 's5',
 's8',
 'a1',
 'a8',
 '80',
 'a7',
 'rsq3',
 'rs3',
 'a4_allroad',
 'q8',
 '100',
 'sq5',
 's4',
 'tt_rs',
 'rs4',
 's7',
 'rs5',
 '90',
 'coupe',
 'a2',
 'tts',
 '920',
 's3',
 '200',
 'v8',
 'e_tron',
 'cr_v',
 'accord',
 'pilot',
 'civic',
 'odyssey',
 'stream',
 'crosstour',
 'elysion',
 'element',
 'ridgeline',
 'fit',
 'hr_v',
 'shuttle',
 'freed',
 'odyssey_na',
 'jazz',
 'z',
 'civic_ferio',
 's2000',
 'cr_z',
 'legend',
 'stepwagon',
 'civic_type_r',
 'fr_v',
 'cr_x',
 'prelude',
 'mobilio_spike',
 'n_box',
 'zest',
 'integra',
 'inspire',
 'vezel',
 'n_wgn',
 'ascot',
 'logo',
 'acty',
 'concerto',
 'avancier',
 'saber',
 'mobilio',
 'crossroad',
 'insight',
 'torneo',
 'airwave',
 'domani',
 'rafaga',


In [101]:
# def get_true(url):
#     if url.split('/')[7] in models:
#         return url
#     else:
#         return None
    
    
    

In [102]:
# df = df.apply(lambda x:get_true(x))

In [103]:
# df.to_csv('filtered_links.csv')

In [284]:
# brand_list = ['BMW',
#               'VOLKSWAGEN',
#               'NISSAN',
#               'MERCEDES',
#               'TOYOTA',
#               'AUDI',
#               'MITSUBISHI',
#               'SKODA',
#               'VOLVO',
#               'HONDA',
#               'INFINITI',
#               'LEXUS']
# vendor_dict = {area: get_vendor(
#     area) for area in brand_list}


def get_vendor(area):
    euro = ['BMW', 'VOLKSWAGEN', 'MERCEDES', 'AUDI', 'SKODA', 'VOLVO']
#     asia = ['NISSAN', 'TOYOTA', 'MITSUBISHI', 'HONDA', 'INFINITI', 'LEXUS']
    if area in euro:
        return 'EUROPEAN'
    else:
        return 'JAPANESE'


def parsing_cars_info(df, url):

    for i in range(0, len(df)):
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        response.encoding = 'utf-8'
        soup = BS(response.text, 'html.parser')

        block = soup.find('ul', class_='CardInfo')

        df['bodyType'] = str(block.find(
            'li', class_='CardInfoRow CardInfoRow_bodytype').a).split('">')[1].split('<')[0]
        df['brand'] = str(url).split('/')[6].upper()
        df['car_url'] = url
#         df['modelDate'] = str(block.find(
#             'li', class_='CardInfoRow CardInfoRow_year').a).split('">')[1].split('<')[0]
        df['color'] = str(block.find(
            'li', class_='CardInfoRow CardInfoRow_color').a).split('">')[1].split('<')[0]
        df['engineDisplacement'] = str(block.find('li', class_='CardInfoRow CardInfoRow_engine').div).split(
            '>')[1].split('/')[0].split(' ')[0] + ' LTR'
        df['enginePower'] = re.findall(
            '\d+', str(block.find('li', class_='CardInfoRow CardInfoRow_engine').div).split('/')[1])[0] + ' N12'
        df['fuelType'] = re.findall(
            '\w+', str(block.find('li', class_='CardInfoRow CardInfoRow_engine').a).split('">')[1].lower())[0]
        df['mileage'] = str(block.find('li', class_='CardInfoRow CardInfoRow_kmAge').find_all('span')[1]).split('>')[1].split('\xa0')[
            0] + str(block.find('li', class_='CardInfoRow CardInfoRow_kmAge').find_all('span')[1]).split('>')[1].split('\xa0')[1]

        df['productionDate'] = re.findall(
            '\d+', str(block.find('li', class_='CardInfoRow CardInfoRow_year').a).split('/')[8])[0]

        df['model_name'] = str(url).split('/')[7].upper()
        df['vehicleTransmission'] = str(block.find('li', class_='CardInfoRow CardInfoRow_transmission').find_all(
            'span')[1]).split('">')[1].split('<')[0]
        df['vendor'] = get_vendor(str(url).split('/')[6].upper())
        df['Владельцы'] = str(block.find('li', class_='CardInfoRow CardInfoRow_ownersCount').find_all(
            'span')[1]).split('>')[1].split('<')[0].replace('\xa0', ' ')
        df['ПТС'] = str(block.find('li', class_='CardInfoRow CardInfoRow_pts').find_all(
            'span')[1]).split('>')[1].split('<')[0]
        df['Привод'] = str(block.find('li', class_='CardInfoRow CardInfoRow_drive').find_all('span')[1]).split('>')[1].split('<')[0]
        df['Руль'] = str(block.find('li', class_='CardInfoRow CardInfoRow_wheel').find_all('span')[1]).split('>')[1].split('<')[0]
        df['Состояние'] = str(block.find('li', class_='CardInfoRow CardInfoRow_state').find_all('span')[1]).split('>')[1].split('<')[0]
        df['Таможня'] = str(block.find('li', class_='CardInfoRow CardInfoRow_customs').find_all('span')[1]).split('>')[1].split('<')[0]
        
        
    return df

In [224]:
str(df['url'][999]).split('/')[7].upper()

'A6'

In [225]:
response = requests.get(df['url'][999], headers={'User-Agent': 'Mozilla/5.0'})
response.encoding = 'utf-8'
soup = BS(response.text, 'html.parser')

block = soup.find('ul', class_='CardInfo')

In [283]:
str(block.find('li', class_='CardInfoRow CardInfoRow_customs').find_all('span')[1]).split('>')[1].split('<')[0]

'Растаможен'

In [285]:
test_df = pd.DataFrame(columns={'bodyType'}, data=[0])

In [286]:
# df['url'][999]

In [287]:
parsing_cars_info(test_df,df['url'][999])


,bodyType,brand,car_url,color,engineDisplacement,enginePower,fuelType,mileage,productionDate,model_name,vehicleTransmission,vendor,Владельцы,ПТС,Привод,Руль,Состояние,Таможня
0,седан,AUDI,https://auto.ru/cars/used/sale/audi/a6/1103497...,белый,1.8 LTR,190 N12,бензин,61446,2017,A6,роботизированная,EUROPEAN,1 владелец,Оригинал,передний,Левый,Не требует ремонта,Растаможен


In [255]:
# НА УДАЛЕНИЕ
['complectation_dict', 'description', 'equipment_dict', 'image',
    'model_info',  'name',  'parsing_unixtime', 'priceCurrency', 'modelDate', 'sell_id', 'super_gen', 'vehicleConfiguration', 'Владение']
# количество дверей потом добавить словарем

['complectation_dict',
 'description',
 'equipment_dict',
 'image',
 'model_info',
 'name',
 'parsing_unixtime',
 'priceCurrency',
 'modelDate',
 'sell_id',
 'super_gen',
 'vehicleConfiguration']